In [1]:
from os.path import isfile, join, exists
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from brainpipe.system import study
from brainpipe.visual import *
from scipy import stats
from itertools import product

#### Plot all info by roi and freq
    DA - # of electrodes with Increases/Decreases - Power change

In [3]:
th, feat = '0.005', 'pow'
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_1000perm_v2/')
npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT.npz')
path_mask = join(path_npz, 'masks_visbrain'+th+'/')
masks_vis_form = join(path_mask, '{}_mask_min{}_{}_minwin{}_th{}.npy')
path2save = join(path_npz,'Signif_elecs'+th+'/')
###############################################################################

subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','MICP','VACJ']
su_codes = ['S0','S1','S2','S3','S4','S5','S6']

# Load mean results by subjects
freqs, win, min_sig = ['0_VLFC','1_delta','2_theta','3_alpha','4_beta','5_gamma1','6_gamma2'], 2, 2 #6_gamma2, 2_theta
for freq in freqs:  
    # Power and classif results
    arch_sig = np.load(npz_form.format('All_subjects',freq, 'odor'))
    if exists(masks_vis_form.format('MAI_RL',min_sig,freq,str(win),th)):
        mask = np.load(masks_vis_form.format('MAI_RL',min_sig,freq,str(win),th))
        mask = np.logical_not(mask) #inverse True False
        su_codes = arch_sig['su_codes'][mask]
        s_Mai_RL = arch_sig['s_MAI_RL'][mask]
        rois_list = np.unique(s_Mai_RL)
        print('masked subjects and rois', su_codes.shape, s_Mai_RL.shape)
        print('rois list',rois_list)
        s_da, s_power = arch_sig['s_da'], (arch_sig['s_elec_pow1']-arch_sig['s_elec_pow0'])*100#/arch_sig['s_elec_pow0'])*100
        s_time = np.arange(-1000,2500,100)
        print(s_da.shape, s_power.shape,s_time.shape)
        #Select the max AUC and corresponding power change for all electrodes
        da_all, pow_all,time_all = np.array([]), np.array([]),np.array([])
        for elec in range(s_da.shape[0]):
            da_elec = s_da[elec]
            idx = [i for i,j in enumerate(da_elec) if j ==max(da_elec)][0]
            pow = s_power[elec][idx]
            da = da_elec[idx]
            time = s_time[idx]
            pow_all = np.hstack((pow_all,pow)) if np.size(pow_all) else pow
            da_all = np.hstack((da_all, da)) if np.size(da_all) else da
            time_all = np.hstack((time_all,time)) if np.size(time_all) else time
        print('global files',da_all.shape, pow_all.shape,time_all.shape)
        #Mask non-significant electrodes
        da_all,pow_all,time_all = da_all[mask], pow_all[mask], time_all[mask]
        print('masked global file',da_all.shape, pow_all.shape,time_all.shape)

        for roi in rois_list:
            #Create an array of mean values by subject for each roi
            su_codes_roi, da_by_su, pow_by_su, time_by_su = np.array([]),np.array([]),np.array([]),np.array([])
            for s,su in enumerate(subjects):
                su_sel = su_codes[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))]
                da_su = np.mean(da_all[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))])
                pow_su = np.mean(pow_all[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))])
                time_su = np.mean(time_all[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))])
                su_codes_roi = np.hstack((su_codes_roi,su_sel)) if np.size(su_codes_roi) else su_sel
                da_by_su = np.hstack((da_by_su,da_su)) if np.size(da_by_su) else da_su
                pow_by_su = np.hstack((pow_by_su,pow_su)) if np.size(pow_by_su) else pow_su
                time_by_su = np.hstack((time_by_su,time_su)) if np.size(time_by_su) else time_su
            #remove nan values
            su_codes_roi = np.unique(su_codes_roi)
            da_by_su = da_by_su[np.logical_not(np.isnan(da_by_su))] 
            pow_by_su = pow_by_su[np.logical_not(np.isnan(pow_by_su))] 
            time_by_su = time_by_su[np.logical_not(np.isnan(time_by_su))]
            #compute all features by roi
            n_patient = len(da_by_su)
            da_mean, da_std = np.mean(da_by_su), np.std(da_by_su)
            n_increase, n_decrease = int(len(pow_by_su[pow_by_su>0])), int(len(pow_by_su[pow_by_su<0]))
            pow_mean_max, pow_std_max = np.mean(pow_by_su[pow_by_su>0]), np.std(pow_by_su[pow_by_su>0])
            pow_mean_min, pow_std_min = np.mean(pow_by_su[pow_by_su<0]), np.std(pow_by_su[pow_by_su<0])
            time_mean, time_std = np.mean(time_by_su), np.std(time_by_su)
            print(freq,roi,'n_patient',n_patient,su_codes_roi,'da',da_mean,da_std,'time',time_mean,time_std)
            print('power',pow_by_su,'increase',n_increase,'decrease',n_decrease,
                  'pow_mean_min',pow_mean_min,'pow_mean_max',pow_mean_max)

            #Plot all information
            info_plot = [n_increase,n_decrease]
            pow_plot, err_pow = [pow_mean_max,pow_mean_min],[pow_std_max,pow_std_min]
            fig = plt.figure()
            gs = gridspec.GridSpec(1, 3,width_ratios=[1,2,2])
            suptitle=fig.suptitle('Bilan roi '+roi+' for '+freq[2:]+' '+str(n_patient)+'patient(s): '+str(su_codes_roi),fontsize=14,y=1.05)
            ax,ax1, ax2 = plt.subplot(gs[0]),plt.subplot(gs[1]),plt.subplot(gs[2])
            ax.bar(np.arange(len([da_mean])),da_mean,yerr=da_std,tick_label=['da'])
            ax.set_ylabel('DA (AUC)')
            ax.set_ylim(0.5)
            ax1.bar(np.arange(len(info_plot)),info_plot,tick_label=['nb+','nb-'])
            ax1.set_ylabel('# of patient')
            ax2.bar(np.arange(len(pow_plot)),pow_plot,yerr=err_pow,tick_label=['mean+','mean-'])
            ax2.set_ylabel('Power change (%)')
            rmaxis(ax, ['right', 'top'])
            rmaxis(ax1, ['right', 'top'])
            rmaxis(ax2, ['right', 'top'])
            ax1.yaxis.set_major_locator(MaxNLocator(3,integer=True))
            ax2.yaxis.set_major_locator(MaxNLocator(5,integer=True))
            fig.tight_layout()
            fig.savefig(path2save+freq[2:]+'_'+roi+'_da_pow_count.png',bbox_inches='tight',bbox_extra_artists=[suptitle])
            plt.clf()
            plt.close()

-> Olfacto loaded
masked subjects and rois (4,) (4,)
rois list ['IFG' 'MTG']
(456, 35) (456, 35) (35,)
global files (456,) (456,) (456,)
masked global file (4,) (4,) (4,)
2_theta IFG n_patient 2 ['S1' 'S4'] da 0.917083333333 0.02125 time 100.0 400.0
power [-422679.07259102   -4318.0766859 ] increase 0 decrease 2 pow_mean_min -213498.574638 pow_mean_max nan


/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2_theta MTG n_patient 2 ['S1' 'S6'] da 0.945 0.00833333333333 time 2100.0 200.0
power [-62143.22354066  -1880.05698285] increase 0 decrease 2 pow_mean_min -32011.6402618 pow_mean_max nan
masked subjects and rois (3,) (3,)
rois list ['MFG']
(456, 35) (456, 35) (35,)
global files (456,) (456,) (456,)
masked global file (3,) (3,) (3,)
3_alpha MFG n_patient 2 ['S0' 'S6'] da 0.920416666667 0.00541666666667 time 750.0 950.0
power [  522.14897038 -2485.50736552] increase 1 decrease 1 pow_mean_min -2485.50736552 pow_mean_max 522.148970376
masked subjects and rois (5,) (5,)
rois list ['SFG']
(456, 35) (456, 35) (35,)
global files (456,) (456,) (456,)
masked global file (5,) (5,) (5,)
4_beta SFG n_patient 2 ['S0' 'S3'] da 0.946369047619 0.0227976190476 time 612.5 487.5
power [ 4643.71437523  -857.18425144] increase 1 decrease 1 pow_mean_min -857.184251436 pow_mean_max 4643.71437523
masked subjects and rois (7,) (7,)
rois list ['MFG' 'OFC']
(456, 35) (456, 35) (35,)
global files (456,) (456,) (45

#### Plot the DA/Pow across time by roi and freq

In [5]:
th = '0.005'
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_1000perm/')
npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT.npz')
path_mask = join(path_npz, 'masks_visbrain'+th+'/')
masks_vis_form = join(path_mask, '{}_mask_min{}_{}_minwin{}_th{}.npy')
path2save = join(path_npz,'Signif_elecs'+th+'/')
###############################################################################

subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','MICP','VACJ']
su_codes = ['S0','S1','S2','S3','S4','S5','S6']

# Load mean results by subjects
freqs, method, win, min_sig = ['0_VLFC','1_delta','2_theta','3_alpha','4_beta','5_gamma1','6_gamma2'],['s_Mail_RL'],2,2
for freq in freqs:  
    # Power and classif results
    arch_sig = np.load(npz_form.format('All_subjects',freq, 'odor'))
    if exists(masks_vis_form.format('MAI_RL',min_sig,freq,str(win),th)):
        mask = np.load(masks_vis_form.format('MAI_RL',min_sig,freq,str(win),th))
        mask = np.logical_not(mask) #inverse True False
        su_codes = arch_sig['su_codes'][mask]
        s_Mai_RL = arch_sig['s_MAI_RL'][mask]
        rois_list = np.unique(s_Mai_RL)
        print('masked subjects and rois', su_codes.shape, s_Mai_RL.shape)
        print('rois list',rois_list)
        s_da, s_power = arch_sig['s_da'][mask], (arch_sig['s_elec_pow1']-arch_sig['s_elec_pow0'])*100#/arch_sig['s_elec_pow0'])*100
        s_power, s_time = s_power[mask],np.arange(-1000,2500,100)
        print(s_da.shape,s_time.shape)
        for roi in rois_list:
            #Create an array of mean da score by subject for each roi
            su_codes_roi, da_by_su,pow_by_su = np.array([]),np.array([]),np.array([])
            for s,su in enumerate(subjects):
                su_sel = su_codes[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))]
                da_su = np.mean(s_da[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))],axis=0)[np.newaxis]
                pow_su = np.mean(s_power[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))],axis=0)[np.newaxis]
                #print('da for one su', da_su.shape)
                su_codes_roi = np.hstack((su_codes_roi,su_sel)) if np.size(su_codes_roi) else su_sel
                if np.isnan(da_su).any()==False:
                    da_by_su = np.vstack((da_by_su,da_su)) if np.size(da_by_su) else da_su
                    pow_by_su = np.vstack((pow_by_su,pow_su)) if np.size(pow_by_su) else pow_su
                #print('da for all', da_by_su.shape)
            su_codes_roi = np.unique(su_codes_roi)
            print('DA by subject',da_by_su.shape, su_codes_roi)
            #compute all features by roi
            n_patient = len(su_codes_roi)
            print(freq,roi,'n_patient',n_patient,su_codes_roi)

            #Plot all information
            fig = plt.figure(figsize=(9,4))
            gs = gridspec.GridSpec(1,2,width_ratios=[1,1])
            suptitle=fig.suptitle('Bilan DA/Pow for '+roi+' in '+freq[2:]+' '+str(n_patient)+'patient(s): '+str(su_codes_roi),fontsize=14,y=1.05)
            ax,ax1 = plt.subplot(gs[0]),plt.subplot(gs[1])
            for s in range(n_patient):
                ax.plot(s_time,da_by_su[s],label=su_codes_roi[s])
                ax1.plot(s_time,pow_by_su[s],label=su_codes_roi[s])
            ax.set_ylim((0.3, 1.))
            ax.set_ylabel('AUC score')
            ax1.set_ylabel('Power change(%)')
            ax.set_xlabel('Time (ms)')
            ax1.set_xlabel('Time (ms)')
            ax.plot(s_time, 0.5*np.ones(len(s_time)), '-', color='grey', linewidth=1)
            ax1.plot(s_time, 0*np.ones(len(s_time)), '-', color='grey', linewidth=1)
            addLines(ax,vLines=[0], vColor=['r'], vShape=['--'], vWidth=[1])
            addLines(ax1,vLines=[0], vColor=['r'], vShape=['--'], vWidth=[1])
            rmaxis(ax, ['right', 'top'])
            rmaxis(ax1, ['right', 'top'])
            ax.legend(loc='best',handletextpad=0.1, frameon=False)
            ax1.legend(loc='best',handletextpad=0.1, frameon=False)
            ax.yaxis.set_major_locator(MaxNLocator(3,integer=True))
            ax1.yaxis.set_major_locator(MaxNLocator(3,integer=True))
            fig.tight_layout()
            fig.savefig(path2save+freq[2:]+'_'+roi+'_da_pow_time.png',bbox_inches='tight',bbox_extra_artists=[suptitle])
            plt.clf()
            plt.close()

-> Olfacto loaded
masked subjects and rois (4,) (4,)
rois list ['IFG' 'MTG']
(4, 35) (35,)
DA by subject (2, 35) ['S1' 'S4']
2_theta IFG n_patient 2 ['S1' 'S4']


/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


DA by subject (2, 35) ['S1' 'S6']
2_theta MTG n_patient 2 ['S1' 'S6']
masked subjects and rois (3,) (3,)
rois list ['MFG']
(3, 35) (35,)
DA by subject (2, 35) ['S0' 'S6']
3_alpha MFG n_patient 2 ['S0' 'S6']
masked subjects and rois (5,) (5,)
rois list ['SFG']
(5, 35) (35,)
DA by subject (2, 35) ['S0' 'S3']
4_beta SFG n_patient 2 ['S0' 'S3']
masked subjects and rois (7,) (7,)
rois list ['MFG' 'OFC']
(7, 35) (35,)
DA by subject (2, 35) ['S0' 'S3']
5_gamma1 MFG n_patient 2 ['S0' 'S3']
DA by subject (2, 35) ['S1' 'S6']
5_gamma1 OFC n_patient 2 ['S1' 'S6']
masked subjects and rois (38,) (38,)
rois list ['MFG' 'OFC' 'SFG']
(38, 35) (35,)
DA by subject (2, 35) ['S0' 'S3']
6_gamma2 MFG n_patient 2 ['S0' 'S3']
DA by subject (2, 35) ['S3' 'S6']
6_gamma2 OFC n_patient 2 ['S3' 'S6']
DA by subject (3, 35) ['S0' 'S3' 'S6']
6_gamma2 SFG n_patient 3 ['S0' 'S3' 'S6']
